## Setup

*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

Set name of spreadsheet to export results to:

In [ ]:
spreadsheet_name = 'feedback_results'

To check that the Kingfisher Collect scraper for your collection completed sucessfully, set the URL of the log file for the job from [Kingfisher Collect](https://collect.kingfisher.open-contracting.org), e.g. `https://collect.kingfisher.open-contracting.org/logs/kingfisher/united_kingdom_contracts_finder_releases/2ef35b40b93b11ec88920c9d92c523cb.log`.

In [ ]:
import getpass

log_url = ''

if log_url != '':

  scrapy_password = getpass.getpass('Enter the password for collect.kingfisher.open-contracting.org:')

Enter your Kingfisher Process credentials:

In [ ]:
user = input('Username:')
password = getpass.getpass('Password:')

Setup notebook environment:

In [ ]:
# Install Kingfisher Colab and required packages
%shell pip install --upgrade 'ocdskingfishercolab<0.4' psycopg2-binary > pip.log

# Install Scrapy Log Analyzer
!pip install git+https://github.com/open-contracting/scrapy-log-analyzer.git

# Import libraries and functions
import pandas as pd
import requests

from google.colab.data_table import DataTable
from ocdskingfishercolab import (
    list_source_ids,
    list_collections,
    render_json,
    set_spreadsheet_name,
    save_dataframe_to_sheet,
    save_dataframe_to_spreadsheet,
    set_search_path)
from pprint import pprint
from scrapyloganalyzer import ScrapyLogFile

# Load https://pypi.org/project/ipython-sql/
%load_ext sql

# Load https://colab.research.google.com/notebooks/data_table.ipynb
%load_ext google.colab.data_table
DataTable.max_columns = 50 # Increase max columns so that dataframes with many columns are rendered as data tables
DataTable.include_index = False # Remove the index from data tables for easier copy-pasting to Google Docs

# Set config
%config SqlMagic.autopandas = True  # Return Pandas DataFrames instead of regular result sets
%config SqlMagic.displaycon = False  # Don't show connection string after execute
%config SqlMagic.feedback = False  # Don't print number of rows affected by DML
set_spreadsheet_name(spreadsheet_name) # Set name of spreadsheet for exporting results

# Connect to database
connection_string = 'postgresql://' + user + ':' + password + '@postgres.kingfisher.open-contracting.org/kingfisher_process?sslmode=require'
%sql $connection_string
  
# Install and setup plotting library
# Maybe this can also be moved to Kingfisher-Colab?
!pip install seaborn

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr

colab_dark_style = {'figure.facecolor': '#383838',
                       'axes.edgecolor': '#d5d5d5',
                       'axes.facecolor': '#383838',
                       'axes.labelcolor': '#d5d5d5',
                       'text.color': '#d5d5d5',
                       'xtick.color': '#d5d5d5',
                       'ytick.color': '#d5d5d5'}

sns.set_style('dark', colab_dark_style)

# Define function to apply number formatting to axis labels
# Maybe this can also be moved to Kingfisher-Colab?
# Needs updating to support other locales
def format_thousands(axis):
  axis.set_major_formatter(tkr.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))

In [ ]:
!